In [5]:
import random
import secrets

In [6]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

In [7]:
def RMPT(p, s): #Assuming number of bits > 600
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

In [ ]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [ ]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

In [ ]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

In [ ]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

In [ ]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

Demo of the attack

Preparation of variables

In [ ]:
p = Generate_Prime(1024, 15)

In [ ]:
q = Generate_Prime(1024, 15)

In [ ]:
N = p * q

In [ ]:
e1=17

In [ ]:
e2=257

In [ ]:
d1 = RSA_generate_D(e1, p, q)

In [ ]:
d2 = RSA_generate_D(e2, p, q)

In [ ]:
message = "For this operation to succeed, both of you need to be mobilized simultaneously. You'll meet your partner once you get there."

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

Encryptions

In [ ]:
encrypted_message_1 = RSA_encrypt(ascii_message, e1, N)

In [ ]:
encrypted_message_2 = RSA_encrypt(ascii_message, e2, N)

Decryptions

In [ ]:
decrypted_message_1 = RSA_encrypt(encrypted_message_1, d1, N)

In [ ]:
decrypted_message_2 = RSA_encrypt(encrypted_message_2, d2, N)

Calculating a1 and a2

In [ ]:
a1=pow(e1,-1,e2)

In [ ]:
a2=int((1-(e1*a1))/e2)

Attempt to decrypt: encrypted_message_1^a1 mod N

In [ ]:
decrypt_attempt_1=(pow(encrypted_message_1,a1,N))

Attempt to decrypt: encrypted_message_2^a2 mod N. Since a2 is negative, we inverse this message to mod N and power it with (-a2)

In [ ]:
encrypt_inverse=pow(encrypted_message_2,-1,N)

In [ ]:
decrypt_attempt_2=(pow(encrypt_inverse,(-a2),N))

Multiply them together

In [ ]:
decrypt_attempt=(decrypt_attempt_1*decrypt_attempt_2)%N

Check decrypted messages of both recipients

In [ ]:
convert_to_string(decrypted_message_1)

In [ ]:
convert_to_string(decrypted_message_2)

Did the eavesdropping succeed?

In [ ]:
convert_to_string(decrypt_attempt)